In [20]:
!pip install pickle-mixin

  Created wheel for pickle-mixin: filename=pickle_mixin-1.0.2-py3-none-any.whl size=5998 sha256=5367dfd88371776c397bff2c1ea6f0cd4a6aef02ecfaa8b3b47f936aba8e516c
  Stored in directory: /Users/sonnguyen2k/Library/Caches/pip/wheels/2a/a4/6c/83bfbc3b94f1bb43d634b07a6a893fd437a45c58b29aea5142
Successfully built pickle-mixin


In [32]:
import re
import pandas as pd
import numpy as np
import pickle

In [6]:
# Install a conda package in the current Jupyter kernel
import sys
!conda install --yes gensim

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: done

## Package Plan ##

  environment location: /Users/sonnguyen2k/opt/anaconda3

  added / updated specs:
    - gensim


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.12.0               |   py38hecd8cb5_0        14.5 MB
    gensim-4.0.1               |   py38h23ab428_0        18.2 MB
    smart_open-5.2.1           |   py38hecd8cb5_0          77 KB
    ------------------------------------------------------------
                                           Total:        32.7 MB

The following NEW packages will be INSTALLED:

  gensim             pkgs/main/osx-64::gensim-4.0.1-py38h23ab428_0
  smart_open         pkgs/main/osx-64::smart_open-5.2.1-py38hecd8cb5_0

The fo

In [33]:
from gensim.models.phrases import Phraser

In [34]:
#process materials:
ev_path = "processors/Englishwords.xlsx"
sf_path =  "processors/Shortform.xlsx"
stopwords_vn_path = "processors/stopwords_vn_dash.txt"
englishwords = pd.read_excel(ev_path, index_col= "English")
shortform = pd.read_excel(sf_path, index_col= "Short")

#phraser for word2vec
bigram = Phraser.load("saves/bigram.pkl")

#word2idx
word2idx = pickle.load(open("saves/word2idx.pickle", "rb"))

In [35]:
print(englishwords)
# print(bigram)
# print(word2idx)

                Vietnamese
English                   
access            truy cập
adapter            cục sạc
ah                       à
ak                       à
app               ứng dụng
...                    ...
try           thử/ cố gắng
website          trang web
wireless         không dây
workshop  buổi diễn thuyết
wow                      ồ

[66 rows x 1 columns]


In [36]:
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

In [37]:
def preprocess(text):
  #bỏ tag html và emoji
  text = re.sub('<[^>]*>', '', text)
  text = deEmojify(text)

  #thay chữ cái viết hoa thành viết thường
  text = text.lower()

  #xóa dấu ngắt câu, xóa link và các chữ có chứa chữ số
  clean_text = []
  punc_list = r'.,;:?!\|/&@`~()-_@#$%^*\'\"'
  for w in (text.split()):
    if "http" in w:
      continue
    clean_text.append(w)
  text = ' '.join(clean_text)
  for punc in punc_list:
    text = text.replace(punc, ' ')

  #xóa bỏ các chữ cái lặp liên tiếp nhau (đỉnhhhhhhhhhh, vipppppppppppppppp)
  length = len(text)
  char = 0
  while char <length-1:
    if text[char] == text[char+1]:
      text = text[:char]+text[char+1:]
      #print(text)
      length-=1
      continue
    char+=1  
  numbers = ["không", "một", "hai", "ba", "bốn", "năm", "sáu", "bảy", "tám", "chín"]
  #chuyển đổi các từ tiếng anh và viết tắt thông dụng sang tiếng Việt chuẩn:
  text_split = text.split()
  for i, w in enumerate(text_split):
    if w in englishwords.index:
      text_split[i] = str(englishwords.loc[w, "Vietnamese"])
    if w in shortform.index:
      text_split[i] = str(shortform.loc[w, "Long"])
    if w.isdigit():
      text_split[i] = ' '.join([numbers[int(c)] for c in w]) 
  text = ' '.join(text_split)

  #loại bỏ tất cả các kí tự đặc biệt còn lại
  digits_and_characters = 'aăâbcdđeêfghijklmnoôơpqrstuưvxywzáàảãạắằẳẵặấầẩẫậéèẻẽẹếềểễệíìỉĩịóòỏõọốồổỗộớờởỡợúùủũụứừửữựýỳỷỹỵ0123456789 '
  text = ''.join([i for i in text if i in digits_and_characters])
  return text


In [38]:
x = preprocess('Phục vụ: nhân viên lễ phép nhiệt tình')
print(x)

phục vụ nhân viên lễ phép nhiệt tình


In [31]:
#split all sentences in corpus
def splitCorpus(corpus):
  t = [sentence.split() for sentence in corpus]
  return t
#join all splited sentences to a big text document
def joinAllSplit(tokenized_sentences):
  sentences = [' '.join(sentence) for sentence in tokenized_sentences]
  return ' '.join(sentences)

#below function get performe preprocessing and remove unknown words
def prepros(sentences):
  new_sentences = [preprocess(sentence) for sentence in sentences]
  splitted_sentences = splitCorpus(new_sentences)
  new = []
  for sentence in bigram[splitted_sentences]:
    new_sentence = ' '.join([word for word in sentence if word in word2idx.keys()])
    new.append(new_sentence)
  return new

#convert words to numbers
def sentenceToInt(sentences):
  #print(sentences)
  int_sentences = []
  for sentence in sentences:
    int_sentence = [word2idx[word] for word in sentence.split()]   
    int_sentences.append(int_sentence)
  return int_sentences

#pad int_sentences to the feature_leng
def padFeature(sentences, feature_leng = 50):
  smatrix = np.zeros((len(sentences), feature_leng))
  for sen_index, sentence in enumerate(sentences):
    padding = max(0, feature_leng - len(sentence))
    for word_index in range(feature_leng):
      if word_index < padding:
        smatrix[sen_index, word_index] = 0
      else:
        smatrix[sen_index, word_index] = sentence[word_index-padding]
  return smatrix

def process(sentences, feature_leng = 50):
  int_sentences = sentenceToInt(sentences)
  feature_matrix = padFeature(int_sentences, feature_leng = 50)
  return feature_matrix


In [ ]:
import torch.nn as nn

#model with 3 part: embedding layer -> stack lstms -> fc layers with softmax classifier
class SentimentLSTM(nn.Module):
    """
    The RNN model that will be used to perform Sentiment analysis.
    """

    def __init__(self, output_size, embedding_dim, hidden_dim, n_layers, n_cell, emb_matrix, drop_prob = 0.2):
        """
        Initialize the model by setting up the layers.
        """
        super().__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        #embedding layer
        self.embedding = nn.Embedding.from_pretrained(emb_matrix, freeze = False)
        # LSTM layers
        self.lstm = nn.LSTM(input_size = embedding_dim,hidden_size = hidden_dim, num_layers = n_layers, batch_first = True, dropout = drop_prob)
        
        # dropout layer
        self.dropout = nn.Dropout(0.3)
        
        # linear and sigmoid layers 
        self.fc = nn.Linear(hidden_dim, output_size)
        #self.fc1 = nn.Linear(hidden_dim, hidden_dim*2)
        #self.relu1 = nn.LeakyReLU()
        #self.fc2 = nn.Linear(hidden_dim*2, output_size)
      
        self.softmax = nn.Softmax(dim = 1)
        

    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        batch_size = x.size(0)
        #print(x)
        # embeddings and lstm_out

        embeds = self.embedding(x)
        embeds = embeds.float()
        #print(type(embeds))
        #print(embeds)
        lstm_out, hidden = self.lstm(embeds, hidden)
        #print(lstm_out.shape)
        #stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        #print(lstm_out.shape)
        
        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        #print(out.shape)
        #out = lstm_out[:, -1, :]
        #print(out.shape)
        out = self.fc(out)
        #out = self.fc1(out)
        #out = self.fc2(out)
        #print(out.shape)
        # sigmoid function
        #print(out.shape)
        out = out.contiguous().view(batch_size, -1, self.output_size)
        out = out[:, -1, :]
        out = self.softmax(out)
        # reshape to be batch_size first
        #print(out.shape)
        #out = out.view(batch_size,n_cell, -1)
        #print(out.shape)
        #out = out[:, -1] # get last batch of labels
        #print(out.shape)
        # return last sigmoid output and hidden state

        return out, hidden
    
    
    def init_hidden(self, batch_size, train_on_gpu = False):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().float(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().float())
        
        return hidden